In [ ]:
import re
import random
import gradio as gr

# Global
l_options = ["bash", "python", "r", "javascript", "java"]
l_mapping = {"bash":"shell", "python":"python", "java":"javascript","javascript":"javascript", "r":"r",}
css = """
#red {background-color: #FA9F9D}
#amber {background-color: #FFD966}
"""



def compare(exp_out, out):
    """
    Compare expected output with output
    """
    if exp_out == out:
        return 
    elif "Error" or "error" in out : 
        return gr.Textbox(value=out,label="Error", elem_id="red")
    else:
        return gr.Textbox(value=out,label="Expected Output does not match Output", elem_id="amber")

    
def Dropdown_lang(dropdown, script):
    """
    Code Languages
    approved language: [('python', 'markdown', 'json', 'html', 'css', 'javascript', 'typescript', 'yaml', 'dockerfile', 'shell', 'r')]
    """
    # return [gr.Code(value=script, language=l_mapping[dropdown], interactive=False), gr.Textbox(value='TypeError out',label="Output")] # test case to be REMOVED
    return [gr.Code(value=script, language=l_mapping[dropdown], interactive=False)]



def timeout_value(timeout):
    """
    Timout value
    """
    return gr.Number(label="Timeout", precision=0, minimum=0, maximum=60)


def converse(x, y, z):
    return z


def reset(z):
    return [], []


def random_response(message, history): # this function will be replaced with the LLM response
    """
    Sample Response - to be deleted
    """
    return random.choice(["Yes", "No"])


with gr.Blocks(css=css, theme=gr.themes.Base(neutral_hue="slate")) as demo:
    gr.Markdown(
        """
        # Welcome to Syntax Gen
        Use this tool to generate and test code using LLMs
        """)
    with gr.Row():
        dropdown = gr.Dropdown(l_options,label='Langauge Selection', value=l_options[3]) # Langauge Selection
        dropdown1 = gr.Dropdown(["Claude", "Llama", "AI21", "Sam"],label="Model Selection") # Model Selection
    # bot = gr.Chatbot(render=False)
    with gr.Row(equal_height=True):
        chat = gr.ChatInterface(random_response, undo_btn=None)
        # dropdown.input(fn=reset, inputs=dropdown, outputs=[bot, chat.chatbot_state]) # Reset the chatbot to clear history automatically
        with gr.Column():
            script = gr.Code(value='console.log(\'Hello World!\');',label="Script", language = l_options[3], interactive=False)
            exp_out = gr.Textbox(value='Hello World!',label="Expected Output")
            out = gr.Textbox(value='Hello World!',label="Output")
            cb = gr.Checkbox(label='Auto-approve and proceed')
            timeout = gr.Number(label="Timeout", precision=0, minimum=0, maximum=60)
            execute = gr.Button(value="Approve and Execute")
    dropdown.change(Dropdown_lang, [dropdown, script], [script, out])
    out.change(compare, [exp_out, out], out)

if __name__ == "__main__":
    demo.launch(debug=True)

    # create events, when chaning model/language selection, it will clear the chat history, scirpt, expected output and output


Running on local URL:  http://127.0.0.1:7861
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://8611e1a09cae179730.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Reset Sample

In [ ]:
import gradio as gr

def converse(x, y, z):
    return z

def reset(z):
    return [], []

# Gradio app
with gr.Blocks() as demo:
    bot = gr.Chatbot(render=False)
    dropdown = gr.Dropdown(["Character 1", "Character 2", "Character 3"], label="Characters", info="Select the character that you'd like to speak to", value="Character 1")
    chat = gr.ChatInterface(
        fn=converse,
        chatbot=bot,
        additional_inputs=dropdown
    )
    dropdown.input(fn=reset, inputs=dropdown, outputs=[bot, chat.chatbot_state])

demo.queue()
demo.launch()